In [159]:
import tensorflow as tf
import numpy as np

In [160]:
english = [['I','am','a','boy'], ['I','am', 'a', 'girl'], ['You', 'are', 'so', 'beautiful']]
korean = [['나','는','소년','이다'], ['나','는','소녀','다'], ['당신','은','너무','아름답다']]
seq_data = np.stack((english, korean), 1)

In [161]:
seq_data = seq_data.tolist()

In [162]:
word_arr = []
for sents in seq_data:
    for seq in sents:
        word_arr += seq
word_arr += ['<S>', '</S>', '<P>']
word2num = {c:i for i, c in enumerate(set(word_arr))}
num2word = {i:c for i, c in enumerate(word2num.keys())}
dic_len = len(word2num)

In [163]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    for i, seq in enumerate(seq_data):
        input = []
        output = []
        target = []
        
        for token in seq[0]:
            input.append(word2num[token])
        input_batch.append(np.eye(dic_len)[input])
        output.append(word2num['<S>'])
        for token in seq[1]:
            output.append(word2num[token])
            target.append(word2num[token])
        target.append(word2num['</S>'])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)

    return input_batch, output_batch, target_batch       

In [164]:
input_batch, output_batch, target_batch = make_batch(seq_data)

In [165]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 500
n_class = n_input = dic_len

In [166]:
tf.reset_default_graph()
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [167]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

In [168]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state = enc_states, dtype=tf.float32)

In [169]:
logits = tf.layers.dense(outputs, n_class, activation=None)

In [170]:
weights = tf.ones(shape=[3,5], dtype=tf.float32)
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=logits, targets=targets, weights=weights))

In [171]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [172]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    if epoch % 50 == 49:
        print('Epoch:', '%03d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
    
print('optimization finished!')

Epoch: 050 cost = 0.049173
Epoch: 100 cost = 0.002945
Epoch: 150 cost = 0.001597
Epoch: 200 cost = 0.000979
Epoch: 250 cost = 0.000284
Epoch: 300 cost = 0.000957
Epoch: 350 cost = 0.000556
Epoch: 400 cost = 0.000500
Epoch: 450 cost = 0.000286
Epoch: 500 cost = 0.000136
optimization finished!


In [173]:
def translate(sentence):
    seq_data = [[sentence.split(' '), len(sentence.split(' '))*['<P>']]]
    input_batch, output_batch, target_batch = make_batch(seq_data)
    prediction = tf.argmax(logits, 2)
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    decoded = [num2word[i] for i in result[0]]
    
    end = decoded.index('</S>')
    translated = ' '.join(decoded[:end])
    
    return translated

In [177]:
print('===translation test===')
print('I am a boy ->', translate('I am a boy'))
print('I am a girl ->', translate('I am a girl'))
print('You are so beautiful ->', translate('You are so beautiful'))
print('I am so beautiful ->', translate('I am so beautiful'))

===translation test===
I am a boy -> 나 는 소년 이다
I am a girl -> 나 는 소녀 다
You are so beautiful -> 당신 은 너무 아름답다
I am so beautiful -> 나 는 소년 이다
